In [ ]:
import requests
import json


# Define the base URL of the API
base_url = "https://api.genius.com"

# Endpoints
artist_endpoint = "/artists/:id"
songs_endpoint = "/artists/:id/songs"

# Replace ':id' with a specific artist ID, for example '16775'
artist_id = "897106"  # This is just an example ID and should be replaced with the actual artist ID needed.
artist_endpoint = artist_endpoint.replace(":id", artist_id)
songs_endpoint = songs_endpoint.replace(":id", artist_id)

# Your bearer token should be added here
bearer_token = "REPLACE"

# Headers with Authorization
headers = {
    "Authorization": f"Bearer {bearer_token}"
}

# Function to get data for a specific artist
def get_artist_data(artist_id):
    response = requests.get(f"{base_url}{artist_endpoint}", headers=headers)
    return response.json()

# Function to get songs for a specific artist
def get_artist_songs(artist_id, sort="title", per_page=20, page=1):
    params = {
        "sort": sort,
        "per_page": per_page,
        "page": page
    }
    response = requests.get(f"{base_url}{songs_endpoint}", headers=headers, params=params)
    return response.json()

# Example usage
# Please note: Without a valid bearer token, these requests will not succeed.
artist_data = get_artist_data(artist_id)
artist_songs = get_artist_songs(artist_id)

# Since we cannot actually hit the API without a valid token, these function calls are commented out.
# You would need to uncomment and provide a valid bearer token for them to work.

pretty_json = json.dumps(artist_songs, indent=4)

print(pretty_json)


In [ ]:
print(type(artist_songs))

In [ ]:
# Parse the JSON
response = json.loads(pretty_json)

# Extract the songs
songs = response["response"]["songs"]

# Extract the api_path of each song
song_data = [{"title": song["full_title"], "artist": song["artist_names"], "api_path": song["api_path"]} for song in songs]

print(song_data)

In [ ]:
# Function to get data for a specific song
def get_song_data(api_path):
    response = requests.get(f"{base_url}{api_path}", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return response.status_code

for i in range(0, len(song_data)):
    song_data[i]["url"] = get_song_data(song_data[i]["api_path"])["response"]["song"]["url"]

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_lyrics_container(url):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the div with the 'data-lyrics-container' attribute
        lyrics_div = soup.find('div', attrs={"data-lyrics-container": True})
        
        # Return the text within the div, or the entire element if needed
        return lyrics_div.get_text() if lyrics_div else None
    else:
        print(f"Failed to retrieve the webpage: Status code {response.status_code}")
        return None

# Example usage:
# Replace 'url' with the actual URL of the website you want to scrape
lyrics = get_lyrics_container('https://genius.com/Central-cee-23-freestyle-demo-lyrics')


for i in range(0, len(song_data)):
    song_data[i]["url"] = get_lyrics_container(song_data[i]["url"])

# Pretty print the list of dictionaries
pretty_string = json.dumps(song_data, indent=4, sort_keys=True)
print(pretty_string)